In [1]:
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

In [2]:
df = pd.read_csv("car_prices.csv", chunksize=10000)

cat_columns = [
    'make', 
    'model', 
    'trim', 
    'body', 
    'transmission', 
    'state', 
    'color', 
    'interior', 
]

numeric_column = [
    'year', 
    'condition', 
    'odometer', 
    'mmr', 
    'sellingprice',
    'days_since_sale'
]

# drop a column




In [3]:
def one_hot_encode(df, columns):
    """
    One-hot encodes specified columns in a pandas DataFrame.
    """
    for column in columns:
        dummies = pd.get_dummies(df[column], prefix=column)
        df = pd.concat([df, dummies], axis=1)
        df.drop(column, axis=1, inplace=True)
    return df

def normalize(df, columns):
    """
    Normalizes specified columns in a pandas DataFrame between 0 and 1.
    """
    for column in columns:
        df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())
    return df

In [4]:




import pandas as pd

total = 0

chunksize = 10000
one_hot_encoded_data = pd.DataFrame()

for chunk in df:

    # Drop the 'vin' column from the chunk
    chunk.drop('vin', axis=1, inplace=True)

    # Convert the 'saledate' column to datetime with timezone
    chunk['saledate'] = pd.to_datetime(chunk['saledate'], utc=True, errors='coerce')

    # Remove the timezone information
    chunk['saledate'] = chunk['saledate'].dt.tz_localize(None)
    
    # Calculate the time since the sale using timezone-naive current datetime
    chunk['days_since_sale'] = pd.Timestamp.now(tz=None) - chunk['saledate']

    # Convert the timedelta to days
    chunk['days_since_sale'] = chunk['days_since_sale'].dt.days

    # Drop the 'saledate' column
    chunk.drop('saledate', axis=1, inplace=True)
    chunk.drop('seller', axis=1, inplace=True)

    # Continue with the rest of the processing
    
    chunk = normalize(chunk, numeric_column)
    chunk = one_hot_encode(chunk, cat_columns)
    
    chunk.fillna(0.0, inplace=True)
    
    one_hot_encoded_data = pd.concat([one_hot_encoded_data, chunk], ignore_index=True)
    
    # print dimensions of the one-hot encoded data
    
    total += len(chunk)
    
    if total > 100000:
        break

one_hot_encoded_data.fillna(0.0, inplace=True)
print(one_hot_encoded_data)

            year  condition  odometer       mmr  sellingprice  days_since_sale  make_Acura  make_Audi  make_BMW  make_Bentley  make_Buick  make_Cadillac  make_Chevrolet  make_Chrysler  make_Dodge  make_FIAT  make_Ferrari  make_Ford  make_GMC  make_Geo  make_HUMMER  make_Honda  make_Hyundai  make_Infiniti  make_Isuzu  make_Jaguar  make_Jeep  make_Kia  make_Land Rover  make_Lexus  make_Lincoln  make_MINI  make_Maserati  make_Mazda  make_Mercedes-Benz  make_Mercury  make_Mitsubishi  make_Nissan  make_Oldsmobile  make_Pontiac  make_Porsche  make_Ram  make_Saab  make_Saturn  make_Scion  make_Subaru  make_Suzuki  make_Toyota  make_Volkswagen  make_Volvo  ...  trim_car signature  trim_cherokee 4x4  trim_cxl  trim_fwd lt  trim_gle  trim_lt w/2lt  trim_sierra 4x2 sl  trim_sub 4x4 v8  trim_xl  model_300ZX  model_Aura Hybrid  model_GS 460  model_Golf GTI  model_MKZ Hybrid  model_Tribute Hybrid  model_g2500  model_golf  model_i-Series  model_tahoe  trim_2+2  trim_2.5 I  trim_2.5 XT  \
0       1.00

In [5]:
training_data = one_hot_encoded_data.sample(frac=0.8, random_state=1)

In [6]:
validation_data = one_hot_encoded_data.drop(training_data.index)

In [7]:
import torch
tensor_x_training = torch.tensor(training_data.drop(columns=['sellingprice'], axis=1).values, dtype=torch.float32)
tensor_y_training = torch.tensor(training_data['sellingprice'].values, dtype=torch.float32)

print(tensor_x_training.shape)
print(tensor_y_training.shape)

torch.Size([88000, 2804])
torch.Size([88000])


In [8]:
import torch

tensor_x_validation = torch.tensor(validation_data.drop(columns=['sellingprice'], axis=1).values, dtype=torch.float32)
tensor_y_validation = torch.tensor(validation_data['sellingprice'].values, dtype=torch.float32)

In [9]:
import torch.optim as optim
from torch import nn
from torchsummary import summary
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch.optim as optim

parameters = [32, 64, 128]
learning_rate = [0.0001, 0.001, 0.01]
patiences = [0, 1, 2]
epochs = [100, 200, 300]
r2_min = {"r2": -1, "parameter": 0, "lr": 0, "patience": 0, "ep": 0}

for parameter in parameters:
    for lr in learning_rate:
        for patience in patiences:
            for ep in epochs:
                
                print(r2_min)
                # sequential layer
                model = nn.Sequential(

                    nn.Linear(2804, parameter),

                    nn.ReLU(),
                    
                    nn.Linear(parameter, parameter),
                    
                    nn.ReLU(),
                    nn.Linear(parameter, parameter),
                    
                    nn.ReLU(),
                    nn.Linear(parameter, parameter),
                    
                    nn.ReLU(),

                    nn.Linear(parameter, parameter),
                    
                    nn.ReLU(),

                    nn.Linear(parameter, parameter),
                    
                    nn.ReLU(),

                    nn.Linear(parameter, parameter),
                    
                    nn.ReLU(),



                    nn.Linear(parameter, 1),

                )

                criterion = nn.MSELoss()

                optimizer = optim.Adam(model.parameters(), lr=lr)

                best_validation_loss = float('inf')
                counter = 0

                for epoch in range(ep):  # maximum number of epochs
                    optimizer.zero_grad()
                    outputs = model(tensor_x_training)
                    loss = criterion(outputs, tensor_y_training.unsqueeze(1))
                    loss.backward()
                    optimizer.step()

                    # Calculate validation loss
                    with torch.no_grad():
                        validation_outputs = model(tensor_x_validation)
                        validation_loss = criterion(validation_outputs, tensor_y_validation.unsqueeze(1))

                    # Check for early stopping
                    if validation_loss < best_validation_loss:
                        best_validation_loss = validation_loss
                        counter = 0
                    else:
                        counter += 1
                        if counter >= patience:
                            break

                #plt.plot(losses)

                from sklearn.metrics import r2_score

                with torch.no_grad():
                    validation_outputs = model(tensor_x_validation)
                    r2 = r2_score(tensor_y_validation.numpy(), validation_outputs.numpy())

                    if r2 > r2_min['r2']:
                        r2_min['r2'] = r2
                        r2_min['parameter'] = parameter
                        r2_min['lr'] = lr
                        r2_min['patience'] = patience
                        r2_min['ep'] = ep

                #print(f"R^2 Score: {r2}")
        
print("stopping")
print(r2_min)


{'r2': -1, 'parameter': 0, 'lr': 0, 'patience': 0, 'ep': 0}
{'r2': 0.0005028620638130032, 'parameter': 32, 'lr': 0.0001, 'patience': 0, 'ep': 100}
{'r2': 0.16000962070408276, 'parameter': 32, 'lr': 0.0001, 'patience': 0, 'ep': 200}
{'r2': 0.16000962070408276, 'parameter': 32, 'lr': 0.0001, 'patience': 0, 'ep': 200}
{'r2': 0.16000962070408276, 'parameter': 32, 'lr': 0.0001, 'patience': 0, 'ep': 200}
{'r2': 0.16000962070408276, 'parameter': 32, 'lr': 0.0001, 'patience': 0, 'ep': 200}
{'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
{'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
{'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
{'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
{'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
{'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
{'r

KeyboardInterrupt: 

In [ ]:
# {'r2': -1, 'parameter': 0, 'lr': 0, 'patience': 0, 'ep': 0}
# {'r2': 0.0005028620638130032, 'parameter': 32, 'lr': 0.0001, 'patience': 0, 'ep': 100}
# {'r2': 0.16000962070408276, 'parameter': 32, 'lr': 0.0001, 'patience': 0, 'ep': 200}
# {'r2': 0.16000962070408276, 'parameter': 32, 'lr': 0.0001, 'patience': 0, 'ep': 200}
# {'r2': 0.16000962070408276, 'parameter': 32, 'lr': 0.0001, 'patience': 0, 'ep': 200}
# {'r2': 0.16000962070408276, 'parameter': 32, 'lr': 0.0001, 'patience': 0, 'ep': 200}
# {'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
# {'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
# {'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
# {'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
# {'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
# {'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
# {'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
# {'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
# {'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
# {'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
# {'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
# {'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
# {'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
# {'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
# {'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
# {'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
# {'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
# {'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
# {'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
# ...
# {'r2': 0.22853186362240463, 'parameter': 32, 'lr': 0.0001, 'patience': 1, 'ep': 300}
# {'r2': 0.3750918919019939, 'parameter': 64, 'lr': 0.0001, 'patience': 0, 'ep': 200}
# {'r2': 0.8372288337703874, 'parameter': 64, 'lr': 0.0001, 'patience': 0, 'ep': 300}
# {'r2': 0.8372288337703874, 'parameter': 64, 'lr': 0.0001, 'patience': 0, 'ep': 300}